# Chapter 8

# Hierarchical Indexing

In [355]:
import pandas as pd
import numpy as np

In [356]:
data = pd.Series(np.random.uniform(size=9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1,2,3,1,3,1,2,2,3]])

In [357]:
data

a  1    0.355743
   2    0.298557
   3    0.663320
b  1    0.131705
   3    0.038897
c  1    0.997216
   2    0.898472
d  2    0.828261
   3    0.338756
dtype: float64

In [358]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

you can slice it!

In [359]:
data['b']

1    0.131705
3    0.038897
dtype: float64

In [360]:
data.loc[['b','d']]

b  1    0.131705
   3    0.038897
d  2    0.828261
   3    0.338756
dtype: float64

In [361]:
data.loc[:,2]

a    0.298557
c    0.898472
d    0.828261
dtype: float64

In [362]:
data.unstack()

,1,2,3
a,0.355743,0.298557,0.663320
b,0.131705,NaN,0.038897
c,0.997216,0.898472,NaN
d,NaN,0.828261,0.338756


The inverse operation of unstack is stack:

In [363]:
data.unstack().stack()

a  1    0.355743
   2    0.298557
   3    0.663320
b  1    0.131705
   3    0.038897
c  1    0.997216
   2    0.898472
d  2    0.828261
   3    0.338756
dtype: float64

In [364]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                    columns=[["Ohio", "Ohio", "Colorado"],
                            ["Green", "Red", "Green"]])


In [365]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [366]:
frame.index.names = ['key1','key2']

In [367]:
frame.columns.names = ['state','color']

In [368]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [369]:
frame.index.nlevels

2

In [370]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [371]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [372]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [373]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

#  Summary Statistics by Level

In [374]:
frame.groupby(level='key2').sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [375]:
frame.T.groupby(level="color").sum()

key1   a      b    
key2   1  2   1   2
color              
Green  2  8  14  20
Red    1  4   7  10

#  Indexing with a DataFrame's columns


In [376]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})


In [377]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [378]:
frame2 = frame.set_index(['c','d'])

In [379]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

by default the comluns used are dropped from thew index. oyu can leave them in with

In [380]:
frame.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

you can reverse the process with reset_index()

In [381]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# Combining and Merging Datasets

#  Database-Style DataFrame Joins

pandas.merge function in pandas is the main entry point for using these algorithms on your data.

In [382]:
df1 = pd.DataFrame({'key': ['b','b','a','c','a','a','b'],
                   'data1': pd.Series(range(7), dtype='int64')})

In [383]:
df2 = pd.DataFrame({'key': ['a','b','d'],
                   'data2': pd.Series(range(3), dtype='Int64')})

In [384]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [385]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [386]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [387]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [388]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

In [389]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [390]:
pd.merge(df3,df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


because c is not shared by both dfs it is excluded unless we use an outer join

In [391]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,a,5.0,0
3,b,0.0,1
4,b,1.0,1
5,b,6.0,1
6,c,3.0,<NA>
7,d,NaN,2


In [392]:
pd.merge(df3,df4, left_on='lkey', right_on='rkey', how='outer')

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [393]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'],
                   'data1': pd.Series(range(6), dtype='Int64')})

In [394]:
df2 = pd.DataFrame({'key':['a','b','a','b','d'],
                   'data2': pd.Series(range(5), dtype='Int64')})

In [395]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [396]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [397]:
pd.merge(df1,df2, on='key', how='left')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [398]:
pd.merge(df1,df2,how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [399]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})

In [400]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})


In [401]:
pd.merge(left,right, on =['key1','key2'], how='outer')

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [402]:
pd.merge(left,right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [403]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


# Merging on Index

In [404]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})

In [405]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [406]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [407]:
right1

,group_val
a,3.5
b,7.0


In [408]:
pd.merge(left1,right1,left_on='key',right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [409]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})

In [410]:
righth_index = pd.MultiIndex.from_arrays(
        [["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
            [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

In [411]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})

In [412]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [413]:
pd.merge(lefth,righth,left_on=['key1','key2'],
            right_index=True, how='outer')

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


you can alos use the indexes of both sides of the merge

In [414]:
left2 = pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],
                    index=['a','c','e'],
                    columns=['Ohio','Nevada']).astype('Int64')

In [415]:
right2 = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13.,14.]],
                      index=['b','c','d','e'],
                      columns=['Missouri','Alabama']).astype('Int64')

In [416]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [417]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [418]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [419]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [420]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [421]:
another = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],
                      index=['a','c','e','f'],
                      columns=['New York','Oregon'])

In [422]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [423]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [424]:
left2.join([right2,another],how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


#  Concatenating Along an Axis

In [425]:
arr = np.arange(12).reshape((3,4))

In [426]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [427]:
#np.concatenate([arr,arr],axis=1)
np.concatenate([arr,arr],axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [428]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")

In [429]:
s2 = pd.Series([2,3,4], index=['c','d','e'], dtype='Int64')

In [430]:
s3 = pd.Series([5,6], index=['f','g'], dtype='Int64')

In [431]:
s1

a    0
b    1
dtype: Int64

In [432]:
s2

c    2
d    3
e    4
dtype: Int64

In [433]:
s3

f    5
g    6
dtype: Int64

In [434]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [435]:
pd.concat([s1,s2,s3], axis='columns')

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [436]:
s4 = pd.concat([s1,s3])

In [437]:
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [438]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [439]:
pd.concat([s1,s4], axis='columns', join='inner')

,0,1
a,0,0
b,1,1


In [440]:
result = pd.concat([s1,s2,s3], keys=['one','two','three'])

In [441]:
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: Int64

In [442]:
result.unstack()

,a,b,c,d,e,f,g
one,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
two,<NA>,<NA>,2,3,4,<NA>,<NA>
three,<NA>,<NA>,<NA>,<NA>,<NA>,5,6


In [443]:
pd.concat([s1,s2,s3], axis='columns', keys=['one','two','three'])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [444]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])

In [445]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])

In [446]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [447]:
df2

,three,four
a,5,6
c,7,8


In [448]:
pd.concat([df1,df2], axis='columns', keys=['level1','level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

you can achieve the same results with

In [449]:
pd.concat({'level1':df1,'level2':df2}, axis='columns')

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [450]:
pd.concat({'level1':df1,'level2':df2}, axis='columns', names=['upper','lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [451]:
df1 = pd.DataFrame(np.random.standard_normal((3,4)),
                  columns=['a','b','c','d'])

In [452]:
df2 = pd.DataFrame(np.random.standard_normal((2,3)),
                  columns=['b','d','a'])

In [453]:
df1

,a,b,c,d
0,0.971291,1.599375,0.413451,0.238959
1,0.654146,0.330528,-0.070590,-1.678646
2,-0.302656,-1.891423,-0.005468,-1.386324


In [454]:
df2

,b,d,a
0,1.688885,-1.665607,1.576322
1,-2.545819,-0.634269,-1.565735


In [455]:
pd.concat([df1,df2], ignore_index=True)

,a,b,c,d
0,0.971291,1.599375,0.413451,0.238959
1,0.654146,0.330528,-0.070590,-1.678646
2,-0.302656,-1.891423,-0.005468,-1.386324
3,1.576322,1.688885,NaN,-1.665607
4,-1.565735,-2.545819,NaN,-0.634269


#  Combining Data with Overlap

In [456]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
         index=['f','e','d','c','b','a'])

In [457]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
             index=['a','b','c','d','e','f'])

In [458]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [459]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [460]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

to line up values by index use:

In [461]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [462]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})

In [463]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})


In [464]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [465]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [466]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


#  Reshaping with Hierarchical Indexing

In [467]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                                     name="number"))


In [468]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [469]:
result = data.stack()

In [470]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [471]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [472]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [473]:
result.unstack(level='state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [474]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")

In [475]:
s2 = pd.Series([4,5,6], index=['c','d','e'], dtype='Int64')

In [476]:
data2 = pd.concat({'one':s1,'two':s2})

In [477]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [478]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [479]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [480]:
data2.unstack().stack(dropna=False)

/var/folders/y_/88_bvhts341bf74krgd3hgch0000gp/T/ipykernel_9605/3936770077.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [481]:
df = pd.DataFrame({"left": result, "right": result + 5}, 
                  columns=pd.Index(["left", "right"], name="side"))

In [482]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [483]:
df.unstack(level='state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [484]:
data = pd.read_csv("examples/macrodata.csv")

In [485]:
data = data.loc[:,['year','quarter','realgdp','infl','unemp']]

In [486]:
data.head()

,year,quarter,realgdp,infl,unemp
0,1959.0,1.0,2710.349,0.00,5.8
1,1959.0,2.0,2778.801,2.34,5.1
2,1959.0,3.0,2775.488,2.74,5.3
3,1959.0,4.0,2785.204,0.27,5.6
4,1960.0,1.0,2847.699,2.31,5.2


In [487]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")

/var/folders/y_/88_bvhts341bf74krgd3hgch0000gp/T/ipykernel_9605/3617454192.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop("year"),


In [488]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [489]:
data.index = periods.to_timestamp('D')

In [490]:
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [491]:
data = data.reindex(columns=['realgdp','infl','unemp'])

In [492]:
data.columns.name = 'item'

In [493]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [495]:
long_data = (data.stack()
            .reset_index()
            .rename(columns={0:'value'}))

In [496]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [499]:
pivoted = long_data.pivot(index='date',columns='item',values='value')

In [501]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [505]:
long_data['value2'] = np.random.standard_normal(len(long_data))

In [506]:
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-0.985249
1,1959-01-01,infl,0.000,-0.060966
2,1959-01-01,unemp,5.800,-0.541769
3,1959-04-01,realgdp,2778.801,1.153803
4,1959-04-01,infl,2.340,-0.876976
5,1959-04-01,unemp,5.100,0.333119
6,1959-07-01,realgdp,2775.488,0.999782
7,1959-07-01,infl,2.740,-0.414243
8,1959-07-01,unemp,5.300,0.114993
9,1959-10-01,realgdp,2785.204,-1.213642


by ommiting the last arguement we can achieve hierarchial columns

In [507]:
pivoted = long_data.pivot(index='date',columns='item')

In [509]:
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -0.060966 -0.985249 -0.541769
1959-04-01  2.34  2778.801   5.1 -0.876976  1.153803  0.333119
1959-07-01  2.74  2775.488   5.3 -0.414243  0.999782  0.114993
1959-10-01  0.27  2785.204   5.6 -1.344771 -1.213642  0.912628
1960-01-01  2.31  2847.699   5.2 -0.857956 -0.595915 -0.793805

In [510]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [513]:
unstacked = long_data.set_index(['date','item']).unstack(level='item') 

In [514]:
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8 -0.060966 -0.985249 -0.541769
1959-04-01  2.34  2778.801   5.1 -0.876976  1.153803  0.333119
1959-07-01  2.74  2775.488   5.3 -0.414243  0.999782  0.114993
1959-10-01  0.27  2785.204   5.6 -1.344771 -1.213642  0.912628
1960-01-01  2.31  2847.699   5.2 -0.857956 -0.595915 -0.793805

#  Pivoting “Wide” to “Long” Format

the inverse of pivot is melt. instead of turning one column into many, it merges many into one producing a DataFrame that is longer than the input. 

In [515]:
df = pd.DataFrame({'key':['foo','bar','baz'],
                  'A':[1,2,3],
                  'B':[4,5,6],
                  'C':[7,8,9]})

In [516]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [517]:
melted = pd.melt(df,id_vars='key')

In [518]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


using pivot we can reshape tpo its original form

In [519]:
reshaped = melted.pivot(index='key', columns = 'variable', values = 'value')

In [520]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [521]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


you can also specify what values you want to include in your pivot

In [522]:
pd.melt(df, id_vars='key', value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


you can also melt with out any group identifiers 

In [523]:
pd.melt(df, value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [ ]:
pd.melt(df, value_vars=["key", "A", "B"])